In [1]:
from shapely.geometry import Point
import seaborn as sns
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

sns.set_style("dark")
#this is geospatial workbook - it might not like seaborn? getting error that it's not found

In [2]:
income_raw=pd.read_excel('../data/income_zip.xlsx')

In [3]:
#This is the crappy imported version - magic happens in the next cell when we create income_clean
income_raw.head(3)

,B19001,Household Income in the Past 12 Months (In 2019 Inflation-adjusted Dollars),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
0,NaN,37013,NaN,37015,NaN,37027,NaN,37062,NaN,37064,...,37221,NaN,37228,NaN,37240,NaN,37243,NaN,37246,NaN
1,NaN,Value,Error,Value,Error,Value,Error,Value,Error,Value,...,Value,Error,Value,Error,Value,Error,Value,Error,Value,Error
2,Total:,35597,617,6673,279,19950,382,4040,188,21640,...,17959,354,1051,240,0,12,0,12,0,12


In [4]:
income_data = pd.read_excel("../data/income_zip.xlsx", skiprows = 1, header=[0, 1], sheet_name = 0)
income_data = income_data[income_data.columns.drop(list(income_data.filter(regex='Error')))]
income_data.columns = income_data.columns.droplevel(1)
income_clean = income_data.set_index('Unnamed: 0_level_0').T.reset_index().rename({"index": "zipcode"}, axis = 1)



In [24]:
#df1.col1 = df1.col1.astype(str)    Convert zipcode column to str (will still say 'object')
income_clean['zipcode']=income_clean['zipcode'].astype(str)
income_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   zipcode               42 non-null     object
 1   Total:                42 non-null     int64 
 2   Less than $10,000     42 non-null     int64 
 3   $10,000 to $14,999    42 non-null     int64 
 4   $15,000 to $19,999    42 non-null     int64 
 5   $20,000 to $24,999    42 non-null     int64 
 6   $25,000 to $29,999    42 non-null     int64 
 7   $30,000 to $34,999    42 non-null     int64 
 8   $35,000 to $39,999    42 non-null     int64 
 9   $40,000 to $44,999    42 non-null     int64 
 10  $45,000 to $49,999    42 non-null     int64 
 11  $50,000 to $59,999    42 non-null     int64 
 12  $60,000 to $74,999    42 non-null     int64 
 13  $75,000 to $99,999    42 non-null     int64 
 14  $100,000 to $124,999  42 non-null     int64 
 15  $125,000 to $149,999  42 non-null     int6

In [31]:
income_clean.head(10)

Unnamed: 0_level_0,zipcode,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"
0,37013,35597,1424,1023,1197,1751,1647,1685,1748,1937,1748,3956,4510,5507,3547,1703,1212,1002
1,37015,6673,421,256,115,415,365,452,350,277,323,719,903,903,639,294,157,84
2,37027,19950,460,173,246,250,254,250,316,538,472,597,1235,1861,2167,1688,2837,6606
3,37062,4040,119,122,138,140,123,124,121,216,140,329,569,645,468,342,286,158
4,37064,21640,771,302,623,512,609,630,721,495,567,1479,1784,2293,2338,2265,2394,3857
5,37072,12533,534,412,456,674,687,506,518,568,544,1095,1311,2186,1120,784,652,486
6,37073,5425,233,151,169,236,290,325,282,184,288,549,557,698,620,262,399,182
7,37075,24275,743,422,553,687,682,986,1058,948,1011,2052,3003,3497,2271,2203,2004,2155
8,37076,17389,698,405,626,496,946,740,912,662,918,1445,2146,2648,1709,1167,1193,678
9,37080,3012,112,119,254,131,188,107,225,101,92,264,262,384,263,161,161,188


## Pulling a list of zip codes from the income dataframe because population dataframe includes all of TN

In [58]:
#create a list of the zip codes from the income dataframe
dav_zips = income_clean["zipcode"].unique()
print(dav_zips)
#this is an array, but the specific zipcodes aren't separated by commas or anything.  This is a problem below, 
#when I try to pull out the population information specifc to these zips using:
#####pop_dav_zips = pop_raw.loc[dav_zips_list, :]

['37013' '37015' '37027' '37062' '37064' '37072' '37073' '37075' '37076'
 '37080' '37082' '37086' '37115' '37122' '37135' '37138' '37143' '37189'
 '37201' '37203' '37204' '37205' '37206' '37207' '37208' '37209' '37210'
 '37211' '37212' '37213' '37214' '37215' '37216' '37217' '37218' '37219'
 '37220' '37221' '37228' '37240' '37243' '37246']


In [59]:
type(dav_zips)

numpy.ndarray

## Bring in the populations

In [8]:
pop_raw=pd.read_csv('../data/zipcode_population.csv')

In [34]:
pop_raw.head(600)
#there are way more zips represented in this dataframe than in income

,zipcode,"lat,long",city,pop
0,37916,"35.953657, -83.933148","Knoxville, Tennessee","11,672"
1,37614,"36.301650, -82.370809","Johnson City, Tennessee","2,036"
2,37212,"36.133973, -86.801471","Nashville, Tennessee","18,547"
3,37351,"35.104194, -85.264615","Lupton City, Tennessee",270
4,37744,"36.182885, -82.740218","Greeneville, Tennessee",372
...,...,...,...,...
595,38380,"35.789889, -88.032881","Sugar Tree, Tennessee",502
596,38486,"35.172506, -87.538424","Westpoint, Tennessee",889
597,38577,"36.564918, -84.922503","Pall Mall, Tennessee","1,236"
598,38318,"35.950509, -88.258084","Buena Vista, Tennessee",513


In [45]:
#Convert pop_raw zipcode column type to str, to match income zipcode type
pop_raw['zipcode']=pop_raw['zipcode'].astype(str)
pop_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   zipcode   610 non-null    object
 1   lat,long  610 non-null    object
 2   city      610 non-null    object
 3   pop       610 non-null    object
dtypes: object(4)
memory usage: 19.2+ KB


In [33]:
#match the column names so they both say zipcode
pop_raw.rename(columns = {'zip':'zipcode'}, inplace=True)

In [18]:
#ended up with an extra index row somewhere
pop_raw=pop_raw.drop(columns=['number'])

In [35]:
pop_raw.head(10)

,zipcode,"lat,long",city,pop
0,37916,"35.953657, -83.933148","Knoxville, Tennessee","11,672"
1,37614,"36.301650, -82.370809","Johnson City, Tennessee","2,036"
2,37212,"36.133973, -86.801471","Nashville, Tennessee","18,547"
3,37351,"35.104194, -85.264615","Lupton City, Tennessee",270
4,37744,"36.182885, -82.740218","Greeneville, Tennessee",372
5,38105,"35.150642, -90.033969","Memphis, Tennessee","8,976"
6,38104,"35.132422, -90.004640","Memphis, Tennessee","24,727"
7,38115,"35.053329, -89.862806","Memphis, Tennessee","40,401"
8,38114,"35.096438, -89.985644","Memphis, Tennessee","34,120"
9,38111,"35.110521, -89.943501","Memphis, Tennessee","45,077"


In [46]:
#confirming there is actually data in the zips which are NaN after the problematic merges:
#df.loc[df['column_name'] == some_value]
pop_raw.loc[pop_raw['zipcode']=='37916']

,zipcode,"lat,long",city,pop
0,37916,"35.953657, -83.933148","Knoxville, Tennessee","11,672"


In [41]:
#Convert pop_raw zipcode column type to object to see what happens when i try and search for a specific zip
pop_raw['zipcode']=pop_raw['zipcode'].astype(object)
pop_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   zipcode   610 non-null    object
 1   lat,long  610 non-null    object
 2   city      610 non-null    object
 3   pop       610 non-null    object
dtypes: object(4)
memory usage: 19.2+ KB


# Merge Attempts

### we need merge the income dataframe with the income dataframe, and we can do a join on the income zipcodes in order to include only davidson county

In [51]:
#merge attempt 1
cases_income = pd.merge(income_clean,pop_raw, on=['zipcode'], how='left')

In [52]:
cases_income.head()

,zipcode,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999",...,"$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more","lat,long",city,pop
0,37013,35597,1424,1023,1197,1751,1647,1685,1748,1937,...,3956,4510,5507,3547,1703,1212,1002,NaN,NaN,NaN
1,37015,6673,421,256,115,415,365,452,350,277,...,719,903,903,639,294,157,84,NaN,NaN,NaN
2,37027,19950,460,173,246,250,254,250,316,538,...,597,1235,1861,2167,1688,2837,6606,NaN,NaN,NaN
3,37062,4040,119,122,138,140,123,124,121,216,...,329,569,645,468,342,286,158,NaN,NaN,NaN
4,37064,21640,771,302,623,512,609,630,721,495,...,1479,1784,2293,2338,2265,2394,3857,NaN,NaN,NaN


In [53]:
#merge_attempt 2
cases_income2 = income_clean.merge(pop_raw, left_on='zipcode', right_on='zipcode', how='left', indicator=True)

In [54]:
cases_income2.head(3)

,zipcode,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999",...,"$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more","lat,long",city,pop,_merge
0,37013,35597,1424,1023,1197,1751,1647,1685,1748,1937,...,4510,5507,3547,1703,1212,1002,NaN,NaN,NaN,left_only
1,37015,6673,421,256,115,415,365,452,350,277,...,903,903,639,294,157,84,NaN,NaN,NaN,left_only
2,37027,19950,460,173,246,250,254,250,316,538,...,1235,1861,2167,1688,2837,6606,NaN,NaN,NaN,left_only


In [55]:
#merge attempt 3
#gdp_and_internet_use=gdp_df.merge(internet_df, on=["Country","Year"], how='outer')
#outer_merged = pd.merge(precip_one_station, climate_temp, how="outer", on=["STATION", "DATE"])

cases_income3 = pd.merge(pop_raw, income_clean, on=["zipcode"])

In [61]:
cases_income3.head(3)

,zipcode,"lat,long",city,pop,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999",...,"$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"


In [53]:
type(dav_zips)

numpy.ndarray

In [54]:
dav_zips_list=dav_zips.tolist()

In [55]:
type(dav_zips_list)

list

In [83]:
#merge attempt 4 - using the dav_zips list
cases_income4 = pd.merge(income_clean, pop_raw, on = dav_zips)

KeyError: array(['37013', '37015', '37027', '37062', '37064', '37072', '37073',
       '37075', '37076', '37080', '37082', '37086', '37115', '37122',
       '37135', '37138', '37143', '37189', '37201', '37203', '37204',
       '37205', '37206', '37207', '37208', '37209', '37210', '37211',
       '37212', '37213', '37214', '37215', '37216', '37217', '37218',
       '37219', '37220', '37221', '37228', '37240', '37243', '37246'],
      dtype=object)

In [57]:
#rows = a_dataframe.iloc[list_of_indexes, :]
pop_dav_zips = pop_raw.loc[dav_zips_list, :]

KeyError: "None of [Index(['37013', '37015', '37027', '37062', '37064', '37072', '37073', '37075',\n       '37076', '37080', '37082', '37086', '37115', '37122', '37135', '37138',\n       '37143', '37189', '37201', '37203', '37204', '37205', '37206', '37207',\n       '37208', '37209', '37210', '37211', '37212', '37213', '37214', '37215',\n       '37216', '37217', '37218', '37219', '37220', '37221', '37228', '37240',\n       '37243', '37246'],\n      dtype='object')] are in the [index]"